## Imports and API

In [246]:
# Libraries
from langgraph.graph import StateGraph
from graphviz import Digraph
from typing import TypedDict, List, Any
from openai import OpenAI
import requests
import feedparser
from dotenv import load_dotenv
import os

client = OpenAI()
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

## ArVix search agent

In [247]:
def search_arxiv(query, max_results=5):
    url = f'http://export.arxiv.org/api/query?search_query={query}&start=0&max_results={max_results}'
    feed = feedparser.parse(requests.get(url).text)
    
    results = []
    for entry in feed.entries:
        results.append({
            'title': entry.title,
            'summary': entry.summary,
            'link': entry.link
        })
    return results


## OpenAlex search agent

In [248]:
def search_openalex(query, max_results=5):
    url = f"https://api.openalex.org/works?search={query}&per-page={max_results}"
    response = requests.get(url)
    data = response.json()
    
    results = []
    for item in data.get('results', []):
        results.append({
            'title': item.get('display_name', 'No Title'),
            'summary': item.get('abstract_inverted_index', {}),
            'link': item.get('id', 'No Link')
        })
    return results


In [249]:
def merge_results(arxiv_results, openalex_results):
    return arxiv_results + openalex_results

## Summerizing agent

In [250]:
def summarize_papers(papers):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = (
                "You are a professional research assistant tasked with analyzing and summarizing academic research papers.\n\n"
                "Carefully review the following research papers and generate a comprehensive general summary that highlights:\n"
                "- The most significant opportunities, advancements, or contributions discussed across the papers.\n"
                "- The key challenges, limitations, or concerns raised collectively in the research.\n"
                "- The most relevant real-world applications, case studies, or practical implications mentioned.\n\n"
                "Write the summary in clear, professional, and concise language suitable for a research-oriented audience.\n"
                "Organize the summary in natural, flowing paragraphs grouped by topic — not by individual papers.\n"
                "Do not list papers separately and do not invent information not found in the content.\n\n"
                "Here are the research papers for your analysis:\n\n"
                f"{content}"
            )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [251]:
def summarize_papers_with_prompt(papers, custom_instruction):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = f"{custom_instruction}\n\nPapers:\n{content}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


## Critic Agent

In [252]:
def critique_summary(summary_text):
    prompt = (
                "You are a critical reviewer of academic research summaries.\n\n"
                "Carefully review the following summarized research papers.\n"
                "For each summary, analyze and provide clear feedback on:\n"
                "- **Completeness:** Are the main opportunities, key challenges or limitations, and real-world applications addressed as per the original research?\n"
                "- **Accuracy:** Does the summary reflect the actual content of the research without exaggeration, omission, or factual errors?\n"
                "- **Clarity and Specificity:** Is the summary clear, specific, and free from vague language or assumptions?\n"
                "- **Missing Details:** Are there important aspects like examples, case studies, or key findings that are missing and should be included?\n\n"
                "If a summary lacks information, suggest exactly what could be added or improved.\n"
                "Your critique should be professional, constructive, and concise.\n"
                "Do NOT rewrite the summary — only provide critique and suggestions.\n\n"
                "Summaries to Review:\n\n"
                f"{summary_text}"
            )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    critique = response.choices[0].message.content
    
    if any(keyword in critique.lower() for keyword in ["missing", "improve", "bias", "error"]):
        return critique, 'needs_rewrite'
    else:
        return critique, 'approved'


## Verifier agent

In [253]:
def verify_summary(summary_text):
    prompt = (
        "Fact-check the following summary. "
        "Highlight any factual inaccuracies or unsupported claims:\n\n"
        f"{summary_text}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [254]:
def compose_final_answer(summary, critique, verification):
    return f"Summary:\n{summary}\n\nCritique:\n{critique}\n\nVerification Notes:\n{verification}"

In [255]:
# state dictionary structure
state = {
    'user_query': '',
    'arxiv_results': [],
    'openalex_results': [],
    'merged_results': [],
    'summary': '',
    'critique': '',
    'verification': '',
    'final_output': '',
    'status': '',
    'rewrite_count': 0
}

In [256]:
class ResearchState(TypedDict):
    user_query: str
    arxiv_results: List[Any]
    openalex_results: List[Any]
    merged_results: List[Any]
    summary: str
    critique: str
    verification: str
    final_output: str
    status: str
    rewrite_count: int

In [257]:
def search_arxiv_node(state: ResearchState) -> ResearchState:
    results = search_arxiv(state['user_query'], max_results=3)
    state['arxiv_results'] = results
    return state

def search_openalex_node(state: ResearchState) -> ResearchState:
    results = search_openalex(state['user_query'], max_results=3)
    state['openalex_results'] = results
    return state

def merge_results_node(state):
    merged = merge_results(state['arxiv_results'], state['openalex_results'])
    state['merged_results'] = merged
    return state

def summarizer_node(state):
    summary = summarize_papers(state['merged_results'])
    state['summary'] = summary
    return state

def critic_node(state: ResearchState) -> ResearchState:
    critique, status = critique_summary(state['summary'])
    state['critique'] = critique
    if state.get('rewrite_count', 0) >= 2:
        state['status'] = 'approved'
    else:
        state['status'] = status
    if status == 'needs_rewrite':
        state['rewrite_count'] = state.get('rewrite_count', 0) + 1
    return state


def verifier_node(state):
    verification = verify_summary(state['summary'])
    state['verification'] = verification
    return state

def final_composer_node(state):
    final = compose_final_answer(state['summary'], state['critique'], state['verification'])
    state['final_output'] = final
    return state

In [258]:
class ResearchState(TypedDict):
    user_query: str
    arxiv_results: List[Any]
    openalex_results: List[Any]
    merged_results: List[Any]
    summary: str
    critique: str
    verification: str
    final_output: str
    status: str

In [259]:
def check_critique_status(state: ResearchState) -> str:
    return state['status']

In [ ]:
graph = StateGraph(ResearchState)

graph.add_node('SearchArxiv', search_arxiv_node)
graph.add_node('SearchOpenAlex', search_openalex_node)
graph.add_node('MergeResults', merge_results_node)
graph.add_node('SummarizerAgent', summarizer_node)
graph.add_node('CriticAgent', critic_node)
graph.add_node('VerifierAgent', verifier_node)
graph.add_node('FinalComposer', final_composer_node)

# Sequential Start
graph.add_edge('__start__', 'SearchArxiv')
graph.add_edge('SearchArxiv', 'SearchOpenAlex')
graph.add_edge('SearchOpenAlex', 'MergeResults')
graph.add_edge('MergeResults', 'SummarizerAgent')
graph.add_edge('SummarizerAgent', 'CriticAgent')

graph.add_conditional_edges('CriticAgent', check_critique_status, {
    'needs_rewrite': 'SummarizerAgent',
    'approved': 'VerifierAgent'
})

graph.add_edge('VerifierAgent', 'FinalComposer')

app = graph.compile()

In [263]:
initial_state = {
    'user_query': "Latest trends in AI and human-agent collaboration",
    'arxiv_results': [],
    'openalex_results': [],
    'merged_results': [],
    'summary': '',
    'critique': '',
    'verification': '',
    'final_output': '',
    'status': ''
}

# Execute the flow (this runs through all nodes automatically)
result = app.invoke(initial_state)

# Print the final output
print(result['final_output'])

Summary:
The research papers collectively highlight several significant opportunities and advancements in the realm of Artificial Intelligence (AI) and its collaborative applications. The emergence of generative AI techniques, such as large language models (LLMs) and text-to-image models, presents new possibilities for enhancing data storytelling by leveraging their capabilities in visual and narrative generation. Additionally, the combination of AI with wireless technologies, as seen in the 3GPP standardization progress, offers the potential to boost the efficiency of wireless transmission and support AI deployments in wireless networks.

However, these advancements are accompanied by key challenges and limitations. The issue of 'Human-AI Collaboration' is critically examined, with concerns raised about the appropriateness of framing interactions with AI as collaborative endeavors. This perspective sheds light on the complexities of AI labor attribution and underscores the need for et